In [0]:
import csv 
import pandas as pd  
from bs4 import BeautifulSoup as soup 
from urllib.request import urlopen as ureq 
import numpy as np
import re
import time
import os

In [0]:
from multiprocessing import cpu_count, Pool

In [0]:
def scrap(x):
  toscrap = 1
  flag=0
  if x==65:
    flag=1
  global s1
  x = chr(x)
  url = "http://howstat.com/cricket/Statistics/Players/PlayerList.asp?Group={}".format(x)
  pagehtml = ureq(url)
  s1 = soup(pagehtml,"lxml") #parse the html
  table = s1.find("table", { "class" : "TableLined" })
  with open('Player.csv', 'a',newline='') as csvfile:
    f = csv.writer(csvfile)
    for x in table:
      if toscrap == 1:
        rows = table.find_all('tr') #find all tr tag(rows)
        for tr in rows:
          data=[]
          d2 = []
          cols = tr.find_all('td') #find all td tags(columns)
          ms=0
          for td in cols:
            data.append(td.text.strip())
            href = td.find_all('a')
            if data[0] == 'Name' and ms == 5:
              if flag == 1:
                data.append('Code')
            for hr in href:
              if hr.has_attr('href') and ms == 4:
                d2.append(hr.attrs['href'][-4: ])
            ms=ms+1
          data = data+d2
          if len(data) == 7:
            if data[4] != '':
              f.writerow(data)
          #print(data[0][0:3])
          if data[0][0:3]=='No.':
            toscrap=0
    

In [0]:
for i in range(65,91):
  scrap(i)
  time.sleep(2)

In [5]:
df = pd.read_csv("Player.csv")
print(df.shape)
del df['Born']
del df['Tests']
del df['ODIs']
del df['T20s']
for i in range(1970,2020):
  df[str(i)] = 0
df.to_csv('Player2.csv', index=False)



(2547, 7)


In [0]:
def calc(df):
  time.sleep(1)
  for i in df.index.values:
    
#     print(df.index.values)
#     print(df.loc[i,'Name'])
    toscrap = 1
    yrs=1970
    total = 0
    k=df.loc[i,'Code']
    url = "http://howstat.com/cricket/Statistics/Players/PlayerYears_ODI.asp?PlayerID={}".format(k)
    pagehtml = ureq(url)
    s1 = soup(pagehtml,"lxml") 
    table = s1.find("table", { "class" : "TableLined" })
    if table:
      for x in table:
        if toscrap == 1:
          rows = table.find_all('tr') 
          for tr in rows:
            data=[]
            cols = tr.find_all('td') 
            for td in cols:
              data.append(td.text.strip())
            if data[0]!='Year' and data[0][0:3]!='Ove':
              for q in range(yrs,int(data[0])+1):
                if data[0] == str(q):
                  total = total+int(data[8])
                  yrs=int(data[0])+1
                df.loc[i,str(q)] = total
            if data[0][0:3]=='Ove':
              toscrap=0
      for l in range(yrs,2020):
        df.loc[i,str(l)] = total
  return df

In [0]:
cores = 10 
partitions = cores 
 
def parallelize(d2, func):
    data_split = np.array_split(d2, partitions)
    pool = Pool(cores)
    d2 = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return d2

In [0]:
d1 = pd.read_csv("Player2.csv")
result = parallelize(d1, calc);
result.head(2547)
del result['Code']
result.to_csv('OdiFinal.csv',index=False)
os.remove('Player2.csv')
os.remove('Player.csv')